In [1]:
!pip install tweepy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.7.9/install/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
consumer_key = "ptdJNRcJY8S2aTUGH8jFqhuBV"
consumer_secret = "GLRjcKyLmEWuF4zdoS297qCJF54WRsflYC1Jutdlh4EiXngUTk"
access_token = "4777023255-pd3o0wDZ1pT1vT03jBFcdx1rGLZn5nqT1DBSMJI"
access_token_secret = "AyjUIyyXhwkDji0j0P4QsUIWxGx8wdPN72ieMkGTFXxOL"

In [3]:
import tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True )


print(api.rate_limit_status())

{'rate_limit_context': {'access_token': '4777023255-pd3o0wDZ1pT1vT03jBFcdx1rGLZn5nqT1DBSMJI'}, 'resources': {'lists': {'/lists/list': {'limit': 15, 'remaining': 15, 'reset': 1613967829}, '/lists/memberships': {'limit': 75, 'remaining': 75, 'reset': 1613967829}, '/lists/subscribers/show': {'limit': 15, 'remaining': 15, 'reset': 1613967829}, '/lists/members': {'limit': 900, 'remaining': 900, 'reset': 1613967829}, '/lists/subscriptions': {'limit': 15, 'remaining': 15, 'reset': 1613967829}, '/lists/show': {'limit': 75, 'remaining': 75, 'reset': 1613967829}, '/lists/ownerships': {'limit': 15, 'remaining': 15, 'reset': 1613967829}, '/lists/subscribers': {'limit': 180, 'remaining': 180, 'reset': 1613967829}, '/lists/members/show': {'limit': 15, 'remaining': 15, 'reset': 1613967829}, '/lists/statuses': {'limit': 900, 'remaining': 900, 'reset': 1613967829}}, 'application': {'/application/rate_limit_status': {'limit': 180, 'remaining': 177, 'reset': 1613967048}}, 'mutes': {'/mutes/users/list': {

In [4]:
def limit_handled(cursor):
    while True:
        try:
            yield next(cursor)
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
        except StopIteration:
            return

In [5]:
search_term = "covid"
tweets = tweepy.Cursor(api.search,
              q=search_term,
              lang="en", wait_on_rate_limit=True, wait_on_rate_limit_notify=True).items(10000)



In [6]:
covid_tweets = []
for tweet in limit_handled(tweets):
    covid_tweets.append(tweet.text)


Rate limit reached. Sleeping for: 67
Rate limit reached. Sleeping for: 780
Rate limit reached. Sleeping for: 780
Rate limit reached. Sleeping for: 780


In [7]:
import numpy as np
np_tweets = np.asarray(covid_tweets)
# np_tweets = np.load("tweets.npy",allow_pickle=True)
np.save("tweets", np_tweets, allow_pickle=True)

In [2]:
!pip install "nltk==3.4.5"

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.7.9/install/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import numpy as np
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

In [4]:
np_tweets = np.load("tweets.npy",allow_pickle=True)
covid_train = np_tweets[:9000]
covid_test = np_tweets[9000:]
print(len(covid_train))
print(len(covid_test))

9000
1000


In [5]:
print(covid_train.shape)
print(covid_train[0])
# print(covid_tweets[0])
len(covid_train)

(9000,)
@KristinRaworth @mtmdiamond The last thing any of us need during covid is rich pasta inspiration. I’ve already move… https://t.co/2DgwmbqTs9


9000

In [6]:
import regex as re
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
import string
from nltk.corpus import stopwords  
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr4/cs640/goutern/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /usr4/cs640/goutern/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr4/cs640/goutern/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
stop_words.add("...")
stop_words.add("rt")
#clean and tokenize the tweets
def clean_tweets(tweets):
    cleaned_tweets = []
    for tweet in tweets:
        cleaned_tweet_tokens = []
        token_tweet = tweet_tokenizer.tokenize(tweet)
        #tag tweets by words for lemmatization normlaization
        for token, tag in pos_tag(token_tweet):
            #regexs to remove punctutation, http,twitter handles, etc...
            token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*(),]|'
                           '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
            token = re.sub("(@[A-Za-z0-9_]+)","", token)
            
            #update tags to work with lemmantizer
            if tag.startswith("NN"):
                pos = 'n'
            elif tag.startswith('VB'):
                pos = 'v'
            else:
                pos = 'a'

            token = lemmatizer.lemmatize(token, pos)
#             print(token)
            #check the token to ensure it is clean and then add it to the cleaned tokens
            if len(token) > 0 and token not in string.punctuation and token.lower() and token not in stop_words:
                cleaned_tweet_tokens.append(token.lower())
        cleaned_tweets.append(cleaned_tweet_tokens)
    return cleaned_tweets

In [18]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.models import KneserNeyInterpolated

tweet_tokens = []
tweet_tokens = (clean_tweets(covid_train))

In [19]:
def ngram_model(ngrams):
    print(len(tweet_tokens))
    train_data, padded_sents = padded_everygram_pipeline(ngrams, tweet_tokens)
    model = KneserNeyInterpolated(ngrams) 
    model.fit(train_data,padded_sents)
    print(model.counts)
    return model   

In [20]:
unigram = ngram_model(1)
bigram = ngram_model(2)
trigram = ngram_model(3)
print(unigram)
print(bigram)
print(trigram)

9000
<NgramCounter with 1 ngram orders and 127553 ngrams>
9000
<NgramCounter with 2 ngram orders and 282106 ngrams>
9000
<NgramCounter with 3 ngram orders and 463659 ngrams>


In [21]:
from datetime import datetime
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
uni_perplex = 0
bi_perplex = 0
tri_perplex = 0
test_tokens = clean_tweets(covid_test)

# bi_test_data, _ = padded_everygram_pipeline(ngrams, tweet_tokens)
# tri_test_data, _ = padded_everygram_pipeline(ngrams, tweet_tokens)


uni_test_data, _ = padded_everygram_pipeline(1, test_tokens)
bi_test_data,_ = padded_everygram_pipeline(2, test_tokens)
# tri_test_data = [nltk.trigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in test_tokens]
tri_test_data, _ = padded_everygram_pipeline(3, test_tokens)

i =0
for test in uni_test_data:
    i += 1
    uni_perplex += unigram.perplexity(test)
uni_perplex = uni_perplex/i
print(uni_perplex)

i =0
for test in bi_test_data:
    try:
        bi_perplex += bigram.perplexity(test)
        i += 1
    except ZeroDivisionError:
        i
bi_perplex = bi_perplex/i
print(bi_perplex)

i =0
for test in tri_test_data:
    try:
        tri_perplex += trigram.perplexity(test)
        i += 1
    except ZeroDivisionError:
        i
tri_perplex = tri_perplex/i
print(tri_perplex)

# for i, test in enumerate(tri_test_data):
#     try:
#         tri_perplex += trigram.perplexity(test)
#     except ZeroDivisonError:
#         print("error")
# tri_perplex = tri_perplex/len(tri_test_data)
# print(tri_perplex)

# for tweet in covid_test:
#     if len(tweet)!=0:
#         uni_perplex += unigram.perplexity(tweet)
#         bi_perplex += bigram.perplexity(tweet)
#         tri_perplex += trigram.perplexity(tweet)

# print(uni_perplex)
# print(bi_perplex)
# print(tri_perplex)
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 23/02/2021 12:35:52
9407.999999999998
1779.3845742485717
56.01130774753686
date and time = 23/02/2021 12:40:41


In [22]:
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
for i in range(10):
    print(unigram.generate(20))
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
for i in range(10):
    print(bigram.generate(20))
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
for i in range(10):
    print(trigram.generate(20))
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 23/02/2021 12:40:41
['apartheid', 'strengthen', 'n95', 'disappointin', 'saying', 'opposite', 'classroom', 'ceaseless', 'initial', 'resid', '#tanzania', 'governments', 'grandchild', 'middle', 'mosque', 'tragedy', 'thru', '#b117', 'garza', 'phoenix']
['epic', 'yr', 'favorite', 'wide', 'old', 'desantis', 'jews', 'improve', '63', 'maintain', 'idea', 'select', '2020', 'not-serious', 'leather', 'harris', '🏻', 'roll', 'near', 'finna']
['dear', 'kate', 'maintaining', 'write', '660', 'first', 'virtual', 'signif', 'stupid', 'prom', 'dt', 'cance', 'glasses-wearers', 'repo', 'subtext', 'andrews', '75', 'dot', 'normalise', '#wearamask']
['backlog', 'corner', 'updated', 'unaccepta', 'state.and', 'presley', 'slowdown', 'maniac', 'strain', '600,000', 'heartfelt', 'patrick', '216', 'dept', 'shirt', 'damn', 'kitchen', 'refusing', 'woodland', '300x']
['protec', 'differ', 'masuk', 'brown', 'va', 'violent', '1,228', '6,553', 'sa', 'bubba', 'chicago', 'sin', '#cancelboards', 'sultan', 'test'

In [23]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr4/cs640/goutern/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [24]:
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
print(covid_train[0])
print(sid.polarity_scores(covid_train[0]))
dt_string = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 23/02/2021 15:15:26
@KristinRaworth @mtmdiamond The last thing any of us need during covid is rich pasta inspiration. I’ve already move… https://t.co/2DgwmbqTs9
{'neg': 0.0, 'neu': 0.708, 'pos': 0.292, 'compound': 0.7906}
date and time = 23/02/2021 15:15:26


In [25]:
from collections import Counter
clean_token_tweets = clean_tweets(np_tweets)

total_compound = 0
negative_tweets = []
positive_tweets = []
for tweet in np_tweets:
    compound = sid.polarity_scores(tweet).get('compound')
    
    #negative tweet
    if compound < -0.05:
        negative_tweets.append(tweet)
    #positive tweet
    elif compound > 0.05:
        positive_tweets.append(tweet)
    total_compound += compound

negative_tokens = clean_tweets(negative_tweets)
positive_tokens = clean_tweets(positive_tweets)

negative_counts = Counter([j for i in negative_tokens for j in i ])
positive_counts = Counter([j for i in positive_tokens for j in i ])

print(negative_counts.most_common(10))
print(positive_counts.most_common(10))

print(total_compound)
print(total_compound/len(clean_token_tweets))

[('rt', 2944), ('…', 2619), ('covid', 2534), ('19', 728), ('the', 643), ('’', 608), ('lose', 547), ('people', 534), ('year', 531), ('500,000', 525)]
[('…', 2326), ('rt', 2180), ('covid', 2014), ('relief', 637), ('19', 629), ('’', 617), ('i', 592), ('vaccine', 295), ('get', 284), ('people', 283)]
-534.9289999999959
-0.05349289999999959
